# 12 高阶pandas

## 12.1 分类数据

In [2]:
import pandas as pd
import numpy as np

In [3]:
# 如果一列经常包含重复值，经常使用unique和value_counts这样的函数
values = pd.Series(['apple', 'pear', 'orange', 'banana'] * 4)
values

0      apple
1       pear
2     orange
3     banana
4      apple
5       pear
6     orange
7     banana
8      apple
9       pear
10    orange
11    banana
12     apple
13      pear
14    orange
15    banana
dtype: object

In [4]:
pd.unique(values)

array(['apple', 'pear', 'orange', 'banana'], dtype=object)

In [5]:
pd.value_counts(values)

apple     4
pear      4
orange    4
banana    4
dtype: int64

In [8]:
# 使用维度表来存储重复数据，维度表包含了不同值，并将主要观测值存储为引用维度表的整数键
# 值
values = pd.Series([0, 1, 0, 0] * 2)
# 维度数据Series
dim = pd.Series(['apple', 'orange'])
dim

0     apple
1    orange
dtype: object

In [9]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [10]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

> 在数据分析时，分类展示会产生显著的性能提升，可以在类别上进行转换的同时不改变代码：
> 1 重命名类别
> 2 在不改变已有的类别顺序的情况下添加一个新的类别

### 12.1.2 pandas中Categorical类型

In [14]:
# Categorical用于承载基于整数的类别展示或编码的数据
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': np.random.randint(3, 15, size=N),
                   'weight': np.random.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,5,1.924889
1,1,orange,11,2.394817
2,2,apple,3,1.792333
3,3,apple,12,0.885257
4,4,apple,6,1.795776
5,5,orange,14,2.776889
6,6,apple,14,3.632011
7,7,apple,9,3.331917


In [16]:
fruit_cat = df.fruit.astype('category')
fruit_cat  # 并不是Numpy数组，而是pandas.Categorical实例

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [17]:
c = fruit_cat.values
type(c)

pandas.core.arrays.categorical.Categorical

In [18]:
# Categorical对象拥有categories和codes属性
c.categories

Index(['apple', 'orange'], dtype='object')

In [19]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [20]:
# 可以通过分配已转换的结果将DataFrame的一列转换为Categorical对西那个
df['fruit'] = df['fruit'].astype('category')
df

,basket_id,fruit,count,weight
0,0,apple,5,1.924889
1,1,orange,11,2.394817
2,2,apple,3,1.792333
3,3,apple,12,0.885257
4,4,apple,6,1.795776
5,5,orange,14,2.776889
6,6,apple,14,3.632011
7,7,apple,9,3.331917


In [21]:
df.fruit.dtype  # categorical对象

CategoricalDtype(categories=['apple', 'orange'], ordered=False)

In [22]:
df.basket_id.dtype  # int64

dtype('int64')

In [23]:
# 可以直接生成pandas.Categorical
my_catefories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_catefories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [24]:
my_catefories.dtype

CategoricalDtype(categories=['bar', 'baz', 'foo'], ordered=False)

In [31]:
# 从另一个数据源获得分类编码数据，可以通过from_codes构造函数创建Categorical
# 默认foo:0, bar:1, baz:2，并且categories中的元素是唯一的，不能存在重复值
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [30]:
# 可以自定义一个有意义的顺序
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_3 = pd.Categorical.from_codes(codes, categories, ordered=True)
my_cats_3

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [32]:
# 对一个未排序的Categorical进行排序
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### 12.1.3 使用Categorical对象进行计算

In [39]:
# 对于使用Categorical和非编码方式的pandas数据整体上是一致的，但使用某些函数，比如groupby，Categorical性能会更好，还有一些函数可以利用ordered标识。
np.random.seed(12345)
draws = np.random.randn(1000)
bins = pd.qcut(draws, 4)
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [46]:
# 指定四分位数名称(不关注分位落在哪个区间)
# 被标记的bins分类数据并不包含数据中箱边界的相关信息
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [47]:
# 可以做汇总统计
bins = pd.Series(bins, name='quartile')
result = (pd.Series(draws).groupby(bins).agg(['max', 'min', 'count']).reset_index())
result

,quartile,max,min,count
0,Q1,-0.685484,-2.949343,250
1,Q2,-0.010115,-0.683066,250
2,Q3,0.628894,-0.010032,250
3,Q4,3.927528,0.634238,250


In [52]:
# 使用分类获得更高的性能
# 如果在特定的数据集上做了大量的分析，将数据转换为分类数据可以产生大幅的性能提升。
# DataFrame中一列的分类版本通常也会明显使用更少的内存
# 下面来看含有1000万元素的Series以及少量的不同类别
N = 10000000
draws_1 = pd.Series(np.random.randn(N))
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

In [53]:
# 将labels转换为Categorical
categories = labels.astype('category')

In [54]:
# 未转换的数据占用了更多的内存
labels.memory_usage()

80000128

In [55]:
categories.memory_usage()

10000332

In [56]:
# 转换需要花费时间，但转换是一次性开销
%time _=labels.astype('category')

CPU times: user 320 ms, sys: 10.8 ms, total: 330 ms
Wall time: 330 ms


> 使用分类对象进行groupby操作明显更快，这是因为底层算法使用了基于整数代码的数组而不是字符串数组

### 12.1.4 分类方法

In [59]:
# 转换为分类对象后，提供了快捷访问类别和代码的方式
s = pd.Series(list('abcd')*2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [61]:
# 查看编码分类
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [62]:
# 可以改变类别集合
actual_categories = list('abcdef')
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (6, object): ['a', 'b', 'c', 'd', 'e', 'f']

In [63]:
# 虽然看起来没有什么改变，但在做数据统计时遵循新的类别
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
f    0
dtype: int64

In [64]:
# 在大型数据集中，可以通过移除未使用的类别来节省内存和提升处理数据的速度
cat_s3 = cat_s[cat_s.isin(['a','b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [65]:
# 会移除未使用的c和d
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

In [66]:
# 生成哑变量
cat_s = pd.Series(list('abcd')*2, dtype='category')
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1
